# Lab 3: Alignment-based metrics in Machine Learning (end)

* Author: Romain Tavenard (@rtavenar)
* License: CC-BY-NC-SA

A lab session from a course on Machine Learning for Time Series at ENSAI.
One can find lecture notes for this course [there](https://rtavenar.github.io/ml4ts_ensai/).

This notebook contains the last section of Lab #3 on "Alignment-based metrics in Machine Learning".

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Data loading

In [ ]:
from tslearn.datasets import CachedDatasets

data_loader = CachedDatasets()

X_train, y_train, X_test, y_test = data_loader.load_dataset("Trace")

# Shapelet models

Shapelet models are time series classifiers that rely on the presence/absence of local shapes in 
time series to make predictions.
These models will be presented in more details later in the course, since they can be seen as a variant of 
shallow convolutional models for time series.

In the following, you will train such models (using their 
[`tslearn` implementation](https://tslearn.readthedocs.io/en/stable/gen_modules/shapelets/tslearn.shapelets.LearningShapelets.html#tslearn.shapelets.LearningShapelets)) 
and visualize their learned decision boundaries.
For the sake of visualization, models made of only 2 local shapes will be learned, and the following 
function will be used to visualize them:

In [ ]:
from matplotlib import cm

def visualize_shapelet_model_2_shapelets(model, X, y):
    distances = model.transform(X)

    plt.figure()
    viridis = cm.get_cmap('viridis', 4)
    # Create a scatter plot of the 2D distances for the time series of each class.
    for i, yi in enumerate(np.unique(y)):
        plt.scatter(distances[y == yi][:, 0],
                    distances[y == yi][:, 1],
                    color=viridis(i / 3),
                    edgecolors='k',
                    label='Class {}'.format(yi))

    # Create a meshgrid of the decision boundaries
    xmin = np.min(distances[:, 0]) - 0.1
    xmax = np.max(distances[:, 0]) + 0.1
    ymin = np.min(distances[:, 1]) - 0.1
    ymax = np.max(distances[:, 1]) + 0.1
    xx, yy = np.meshgrid(np.arange(xmin, xmax, (xmax - xmin) / 200), 
                         np.arange(ymin, ymax, (ymax - ymin) / 200))
    
    weights, biases = model.get_weights('classification')
    Z = []
    for x, y in np.c_[xx.ravel(), yy.ravel()]:
        Z.append(np.argmax([biases[i] + weights[0][i]*x + weights[1][i]*y
                            for i in range(4)]))
    Z = np.array(Z).reshape(xx.shape)
    plt.contourf(xx, yy, Z / 3, cmap=viridis, alpha=0.25)

    plt.legend()
    plt.xlabel('$d(\mathbf{x}, \mathbf{s}_1)$')
    plt.ylabel('$d(\mathbf{x}, \mathbf{s}_2)$')
    plt.xlim((xmin, xmax))
    plt.ylim((ymin, ymax))
    plt.title('Distance transformed time series')
    plt.show()

Let us start with a first attempt.
The following code snippet defines a model made of 2 shapelets (=local shapes to be learnt) and train
them (as well as the corresponding linear decision boundaries) for 500 epochs:

In [ ]:
from tslearn.shapelets import LearningShapelets

shp_clf = LearningShapelets(
    n_shapelets_per_size={20: 2},  # 2 shapelets of length 20
    max_iter=500,
    verbose=0,                     # Do not print information on the evolution of the loss
    scale=False,
    random_state=42
)
shp_clf.fit(X_train, y_train)

**Question #8.** Visualize the obtained decision boundaries. 
Would you expect this model to be a decent classifier?

**Question #9.** Shapelet models usually benefit from scaling of the input time series 
(yet this is, of course, very dataset-specific).
See if this helps in our case.

**Question #10.** By default, the `tslearn` implementation uses SGD as an optimizer.
See if using `"adam"` helps in our case.

**Question #11.** Set the learning rate of the Adam optimizer to 1e-2 
(you will need to import the `Adam` class from `keras` as follows) and see if this improves the learning.

In [ ]:
from keras.optimizers import Adam

